In [1]:
import os
import pandas as pd
import numpy as np

from cube.api import Cube
from config import PATH_TO_SAVE_PROCESSED_FILES, PATH_TO_SAVE_GENERAL_DF
from utils.text_data_transformation import transform_raw_data
from utils.dialog_manipulation import prepare_dialogs

In [2]:
# 0) Firstly, use data_transformation.py to get "subdialogs_{id}.csv" files


# or DIALOG_IDS = [-1] for all in dir
DIALOG_IDS = [-1]
# DIALOG_IDS = ["138918380", "470409323", "347963763", "850978724", "511986933"]
path_to_save_result = os.path.join(PATH_TO_SAVE_PROCESSED_FILES, "subdialog_keywords.csv")

# flag_get_all = 0
# if DIALOG_IDS[0] == -1:
#     DIALOG_IDS = os.listdir(path_to_prepared_dialogs)
#     flag_get_all = 1

frames = []
df_idf = pd.DataFrame()
general_n_subdialogs = 0


data_for_df = []
data = pd.read_csv(PATH_TO_SAVE_GENERAL_DF)

n_subdialog = 0
dialog_dict, dialog_text = {}, ''
for index, row in data.iterrows():
    if n_subdialog != row.subdialog_id:
        n_subdialog += 1
        general_n_subdialogs += 1
        if n_subdialog != 1:
            dialog_dict["text"] = dialog_text
            data_for_df.append(dialog_dict)

        dialog_dict = {}
        dialog_text = ''
        dialog_dict["dialog_id"] = general_n_subdialogs

    if not pd.isnull(row.preprocessed_message):
        dialog_text += " " + row.preprocessed_message

df_idf = pd.DataFrame(data_for_df)
df_idf

,dialog_id,text
0,1,​50 русскоязычных YouTube-каналов о языках пр...
1,2,​JavaScript для Python-программистов: познани...
2,3,​Как в Google Chrome измерить использование п...
3,4,интенсив практикум penetration testing получ...
4,5,python проект оказать огромный влияние разра...
...,...,...
4718,4719,просити передзвонити
4719,4720,Подчеркни недостатки
4720,4721,случайная карточка
4721,4722,случайная карточка


In [3]:
df_idf['text'].replace('', np.nan, inplace=True)
df_idf.dropna(subset = ["text"], inplace=True)

# if you have untransformed_data so uncomment it !!!!!!!!!!!!!
# if lang == "ua":
#     cube = Cube(verbose=True)
#     cube.load("uk")
# df_idf['text'] = df_idf['text'].apply(lambda x: transform_raw_data(x, lang, "words_frequency", cube))

#show the first 'text'
df_idf['text'][2]

' \u200bКак в Google Chrome измерить использование памяти веб-страницей\n\nДержим руку на пульсе производительного веба. В этой статье учимся измерять утечки памяти страницы Google Chrome с помощью нового интерфейса performance.measureMemory().\n\nhttps://proglib.io/w/4e1c965a'

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
import re

def get_stop_words():
    """load stop words """

    try:
        with open(os.path.join(os.getcwd(), "..", "dicts", "ukrainian_stopwords.txt"), "r", encoding="utf-8") as file:
            stop_words = str(file).strip().split()

    except FileNotFoundError:
        with open(os.path.join(os.getcwd(), "dicts", "ukrainian_stopwords.txt"), "r",
                  encoding="utf-8") as file:
            stop_words = str(file).strip().split()

    return stop_words

#load a set of stop words
stopwords=get_stop_words()

#get the text column
docs=df_idf['text'].tolist()

#create a vocabulary of words,
#ignore words that appear in 85% of documents,
#eliminate stop words
cv=CountVectorizer(max_df=0.85,stop_words=stopwords)
word_count_vector=cv.fit_transform(docs)

d:\python\envs\venv_telegram_analysis\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['_io', 'analysis', 'dialogs', 'dicts', 'encoding', 'mode', 'my_work', 'name', 'programming', 'telegram', 'textiowrapper', 'txt', 'ukrainian_stopwords', 'utf'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [5]:
cv=CountVectorizer(max_df=0.85,stop_words=stopwords,max_features=10000, ngram_range=(2, 2))
word_count_vector=cv.fit_transform(docs)
word_count_vector.shape

(4220, 10000)

In [6]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

# get text docs into a list
docs_text=df_idf['text'].tolist()

In [7]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""

    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]

        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]

    return results

# you only needs to do this once
feature_names=cv.get_feature_names()

# get the document that we want to extract keywords from
doc=docs_text[1]

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,10)

# now print the results
print("\n=====Text=====")
print(docs_text[1])
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


=====Text=====
 ​JavaScript для Python-программистов: познание через сравнение

Если вы ранее не использовали JavaScript, но знакомы с Python, этот лонгрид поможет вашему знакомству. Имеющиеся знания помогут пропустить очевидные вещи и быстрее войти в тему. 

https://proglib.io/w/2434f8cd ⚡️ 15 – 17 сентября приглашаем на онлайн-конференцию ESCAPE (Essential Skills, Competencies and People Engineering). ⚡️Это нетехническая конференция для всех, кто работает в IT и первая конференция EPAM, посвященная Essential skills.

✍🏼 Три дня мы будем делиться полезной информацией о том, что делает инженеров и руководителей профессионалами, помимо технологий.

👨🏻‍💻Вас ждут доклады и воркшопы про обучение, осознанность, коммуникацию. 
 
🤝Вы узнаете, как эффективно общаться с заказчиками и с людьми разных культур, как построить работу в распределенной команде, как справиться с профессиональным выгоранием. Поговорим и на другие важные темы.  

🚀 Зарегистрироваться и посмотреть программу конференции м

In [8]:
# put the common code into several methods
def get_keywords(idx):

    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([docs_text[idx]]))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)

    return keywords

def print_results(idx,keywords):
    # now print the results
    print("\n=====Text=====")
    print(docs_text[idx])
    print("\n===Keywords===")
    for k in keywords:
        print(k,keywords[k])

In [9]:
#generate tf-idf for all documents in your list. docs_text has 500 documents
tf_idf_vector=tfidf_transformer.transform(cv.transform(docs_text))

results=[]
for i in range(tf_idf_vector.shape[0]):

    # get vector for a single document
    curr_vector=tf_idf_vector[i]

    #sort the tf-idf vector by descending order of scores
    sorted_items=sort_coo(curr_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)

    results.append(keywords)

df=pd.DataFrame(zip(docs,results),columns=['doc','keywords'])
for result in results:
    result2 = {k: v for k, v in sorted(result.items(), key=lambda item: item[1], reverse=True)}

df.to_csv(path_to_save_result, index=False)
print("result df")
df

result df


,doc,keywords
0,​50 русскоязычных YouTube-каналов о языках пр...,"{'https proglib': 0.49, 'proglib io': 0.329, '..."
1,​JavaScript для Python-программистов: познани...,"{'кто работает': 0.29, 'знания помогут': 0.29,..."
2,​Как в Google Chrome измерить использование п...,"{'google chrome': 0.889, 'https proglib': 0.38..."
3,интенсив практикум penetration testing получ...,"{'цикл python': 0.225, 'программа операционный..."
4,python проект оказать огромный влияние разра...,"{'python проект': 0.677, 'proglib io': 0.388, ..."
...,...,...
4215,просити передзвонити,{}
4216,Подчеркни недостатки,{}
4217,случайная карточка,{'случайная карточка': 1.0}
4218,случайная карточка,{'случайная карточка': 1.0}
